In [1]:
import os.path
import re
import selenium
import pandas as pd
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions
import json
from collections import OrderedDict
import sys

In [2]:
wd = "./chromedriver"
driver = webdriver.Chrome(wd)

In [3]:
driver.get('https://news.naver.com/main/list.nhn?mode=LPOD&mid=sec&oid=028&date=20190728')
today_html = driver.page_source
listup = BeautifulSoup(today_html, 'lxml')

In [4]:
news_list = listup.find_all('a', {'class' : 'nclicks(cnt_papaerart)'})
news_list += listup.find_all('a', {'class' : 'nclicks(cnt_papaerart3)'})
news_list += listup.find_all('a', {'class' : 'nclicks(cnt_papaerart4)'})
news_list += listup.find_all('a', {'class' : 'nclicks(cnt_flashart)'})

In [5]:
for index in range(len(news_list)):
    
    addr = news_list[index]['href']
    driver.get(addr)
    
    # 스포츠 뉴스와 연예 뉴스는 제외 (형식도 다르고 목적과 맞지 않음.)
    check = driver.current_url
    if ('sports' in check) or ('entertain' in check):
        continue
        
    html = driver.page_source

    # 타이틀, 분류, 날짜, 언론사, 내용, 댓글 
    dom = BeautifulSoup(html, 'lxml')
    
    category_raw = dom.find('em', {'class' : 'guide_categorization_item'})
    category = category_raw.text

    title_raw = dom.find_all('h3', {'id' : 'articleTitle'})
    title = [title.text for title in title_raw]
    title = str(title[0])
    
    # 저장시 문제 안생기게
    title = re.sub("\"",'',title)
    title = re.sub("\'",'',title)
    title = re.sub("‘",'',title)
    title = re.sub("’",'',title)
    title = re.sub(" ",'',title)
    title = re.sub("\?",'',title)
    title = re.sub(":",'',title)
    title = re.sub("\/",'',title)

    date_raw = dom.find_all('span', {'class' : 't11'})
    date = date_raw[0].text.split()[0]

    press_raw = dom.find('div', {'class' : 'press_logo'})
    press = press_raw.select('a')[0].find('img')['title']

    category_raw = dom.find('em', {'class' : 'guide_categorization_item'})
    category = category_raw.text

    contents_raw = dom.find('div', {'id' : 'articleBodyContents'})
    contents = contents_raw.text

    # 네이버 뉴스에는 아래와 같은 주석이 항상 있음. 이 주석을 제거하기 위한 코드
    # \n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}\n\n 
    clean_index = contents.index('removeCallback') + 23
    contents = contents[clean_index :]


    # 기사 포맷이 거의 항상 아래와 같음. 필요 없는 정보를 제거하기 위한 코드
    # [ⓒ한겨레신문 : 무단전재 및 재배포 금지]
    if '재배포' in contents:
        reporter_index = contents.index('재배포') - 15
        contents = contents[:reporter_index]

    time.sleep(0.2)
    pages = 0
    
    try:
        driver.find_element_by_css_selector(".u_cbox_in_view_comment").click()
        time.sleep(0.2)
    except exceptions.ElementNotInteractableException as e:
        pass
    except exceptions.NoSuchElementException as e:
        try:
            new_addr = dom.find_all('div', {'class' : 'simplecmt_links'})
            new_addr = new_addr[0].select('a')[0]['href']
            driver.get(new_addr)
            time.sleep(0.2)
        except:
            pass
        pass
    
    try:
        driver.find_element_by_css_selector(".u_cbox_sort_label").click()
        time.sleep(0.2)
    except exceptions.NoSuchElementException as e:
        pass
 
    i = 0

    try:
        while i < 30 :
            driver.find_element_by_css_selector(".u_cbox_btn_more").click()
            time.sleep(0.1)
            i+=1

    except exceptions.ElementNotVisibleException as e: # 페이지 끝
        pass

    except Exception as e: # 다른 예외 발생시 확인
        print(e)

    html = driver.page_source
    dom = BeautifulSoup(html, 'lxml')
    
    comments_raw = dom.find_all('span', {'class' : 'u_cbox_contents'})
    comments = [comment.text for comment in comments_raw]
    
    if (len(comments)<1):
        continue

    like_comments_raw = dom.find_all('em', {'class' : 'u_cbox_cnt_recomm'})
    like_comments = [int(like.text) for like in like_comments_raw]

    hate_comments_raw = dom.find_all('em', {'class' : 'u_cbox_cnt_unrecomm'})
    hate_comments = [int(hate.text) for hate in hate_comments_raw]
    
    df = pd.DataFrame({'comment' : comments, 'like' : like_comments, 'dont_like' : hate_comments})
    df = df.sort_values(by = 'like', ascending = False)
    
    file_name = './'+ category+ '/' + press+'_'+date+'_'+title + '.json'

    file_data = OrderedDict()
    file_data['contents'] = contents
    file_data['comment'] = df['comment'].values.tolist()
    file_data['like'] = df['like'].values.tolist()
    file_data['dont_like'] = df['dont_like'].values.tolist()
    
    directory = './' + category

    if os.path.exists(directory):
        with open(file_name, 'w', encoding = 'utf-8') as make_file:
            json.dump(file_data, make_file, ensure_ascii=False, indent='\t')

    else:
        os.mkdir(directory)
        with open(file_name, 'w', encoding = 'utf-8') as make_file:
            json.dump(file_data,  make_file,ensure_ascii=False, indent='\t')


Message: no such element: Unable to locate element: {"method":"css selector","selector":".u_cbox_btn_more"}
  (Session info: chrome=75.0.3770.142)

Message: element not interactable
  (Session info: chrome=75.0.3770.142)

Message: element not interactable
  (Session info: chrome=75.0.3770.142)

Message: element not interactable
  (Session info: chrome=75.0.3770.142)

Message: element not interactable
  (Session info: chrome=75.0.3770.142)

Message: element not interactable
  (Session info: chrome=75.0.3770.142)

Message: element not interactable
  (Session info: chrome=75.0.3770.142)

Message: no such element: Unable to locate element: {"method":"css selector","selector":".u_cbox_btn_more"}
  (Session info: chrome=75.0.3770.142)

Message: no such element: Unable to locate element: {"method":"css selector","selector":".u_cbox_btn_more"}
  (Session info: chrome=75.0.3770.142)

Message: element not interactable
  (Session info: chrome=75.0.3770.142)

Message: element not interactable
  (S

In [9]:
with open('./정치/한겨레_2019.07.28._넉달째맹탕국회…정치가없다.json', encoding = 'utf-8') as f:
    data = json.load(f)
    df = pd.DataFrame({'comment' : data['comment'], 'like':data['like'], 'dont_like':data['dont_like']})

In [10]:
data['contents']

'의 115일간 안 열려… 올해 쟁점법안 1건도 처리 못해 한국당 도 넘은 ‘노딜 전략’, 민주당도 절박함 없어 “총선 향한 여야, 적대적 공생”28일 오후 국회 모습. 김경호 선임기자 jijae@hani.co.kr  나라 안팎의 악재들이 겹치며 해결이 시급한 현안이 쌓여가고 있지만, 정작 입법과 예산의 ‘키’를 쥐고 있는 국회는 최악의 한 해를 보내고 있다. 29일 7월 임시국회 소집이 예고돼 있지만, 추가경정예산안 처리와 ‘원포인트 안보 국회’에 이견이 있는 여야는 주말 사이 어떤 의사일정도 잡지 못했다. 국회의 책임 방기가 도를 넘었다는 비판이 거세다.  올해 국회가 법안 처리를 위해 본회의를 연 날은 단 사흘이다. 통과된 법안들은 모두 비쟁점 법안들로, 여야 간 이견이 있는 쟁점 법안은 한 건도 국회를 통과하지 못했다. 내년 총선을 의식해 지지층 결집에만 집착하는 정치지형 등이 원인으로 꼽힌다. 일부에선 ‘강 대 강’ 대치 국면, 여야의 적대적 공생이 굳어지고 있다는 우려마저 나온다. ( ▶관련기사 보기 : ‘총선용’ 트집잡고 법사위서 발목…민생·경제법안 ‘올스톱’ )  ■ 법안 성사 ‘0’인 상임위 5곳  28일 현재 국회 상임위원회 17곳 중 올해 들어 법안 제·개정을 1건도 해내지 못한 상임위가 5곳이나 된다는 점은 국회가 얼마나 본업에 손을 놓고 있었는지 상징적으로 보여준다. 5곳은 과학기술정보방송통신위원회·외교통일위원회·문화체육관광위원회·정보위원회·여성가족위원회다.  본회의 통과 법안 수도 참담한 수치다. 총선이 있었던 2016년(4~12월) 720건을 처리한 20대 국회는 2017년에 2121건, 2018년에 2723건의 법안을 처리했다. 올해 들어서는 200일이 넘도록 425건을 처리하는 데 그쳤다. 사실상 마비 상태인 셈이다. 지난 4월5일 마지막 본회의가 열린 뒤 115일이 흘렀다.  사정이 이렇다 보니 4월25일 국회에 제출된 추경안은 95일째 표류 중이다. 2조3천억원 규모였던 2000년 추경안(106일) 이후 역대 두번째 장기 표류

In [11]:
df.head(100)

,comment,like,dont_like
0,115일치 월급삭감에 동의하시는분??,14,0
1,국회의원 필요없는거 아닙이까?,12,0
2,국회의원 때문에 나라가 갈렸고발전이 없다.우린 4년마다 왜 저런 사람들의 쇼에 놀아...,12,0
3,노는건좋다 월급은반납해라 뭐하는짓거리들인지,11,0
4,국회 해산합시다. 일히지않은 국회의원들 세비도 주지말아야합니다. 무노동 ...,11,0
5,법을 고쳐라. 국회의원 직접선거. 임기보장. 특권. 9천억의 세비지출. 모두...,11,0
6,"국회안열어도, 나라가 기울어도 자기네는 먹고살만하다 이거지..국회의원한테는 출퇴근용...",11,0
7,자한당 전략은 나라가 어찌되든 어떻게든 문정권 방해하는 것 뿐,10,0
8,무조건 반대만 하는 자한당.. 토론도 안하는 자한당... 그리고 죽어라 찍어대는 갱...,9,1
9,이게 자한당 전략이다. 북한 핑계로 한 국회무력화. 결국 일본에 대한 그 어떤 조치...,9,0
